#   

# 作業目標:

    1. 藉由固定的 dataset, 來驗證不同loss function
    2. Dataset 的特性跟我們選用的loss function 對accrancy 的影響
    
    
# 作業重點: 
    請分別選用 "MSE", "binary _crossentropy"
    查看Train/test accurancy and loss rate
    

# 導入必要的函數

In [1]:
from keras.datasets import cifar10
import numpy as np
np.random.seed(10)

Using TensorFlow backend.


# 資料準備

In [2]:
#取得Keras Dataset
(x_img_train,y_label_train),(x_img_test,y_label_test)=cifar10.load_data()

In [3]:
#確認 CIFAR10 Dataset 資料維度
print("train data:",'images:',x_img_train.shape,
      " labels:",y_label_train.shape) 
print("test  data:",'images:',x_img_test.shape ,
      " labels:",y_label_test.shape) 

train data: images: (50000, 32, 32, 3)  labels: (50000, 1)
test  data: images: (10000, 32, 32, 3)  labels: (10000, 1)


In [4]:
#資料正規化
x_img_train_normalize = x_img_train.astype('float32') / 255.0
x_img_test_normalize = x_img_test.astype('float32') / 255.0

In [5]:
#針對Label 做 ONE HOT ENCODE
from keras.utils import np_utils
y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)
y_label_test_OneHot.shape

(10000, 10)

# 建立模型

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

In [27]:
model = Sequential()

In [37]:
#卷積層1

In [28]:
model.add(Conv2D(filters=32,kernel_size=(3,3),
                 input_shape=(32, 32,3), 
                 activation='relu', 
                 padding='same'))
model.add(Dropout(rate=0.25))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), 
                 activation='relu', padding='same'))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(rate=0.25))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(10, activation='sigmoid'))
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_13 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_14 (Dropout)         (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 4096)             

# 載入之前訓練的模型

In [21]:
try:
    model.load_weights("SaveModel/cifarCnnModel.h5")
    print("載入模型成功!繼續訓練模型")
except :    
    print("載入模型失敗!開始訓練一個新模型")

載入模型失敗!開始訓練一個新模型


# 訓練模型

In [19]:
#loss = categorical_crossentropy
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
train_history = model.fit(x=x_img_train_normalize,y=y_label_train_OneHot,validation_split=0.2,epochs=10,batch_size=200,verbose=2)

# '''
# 作業:
# 請分別選用 "MSE", "binary _crossentropy"
# 查看Train/test accurancy and loss rate
# '''

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
 - 75s - loss: 1.5674 - accuracy: 0.4342 - val_loss: 1.3595 - val_accuracy: 0.5479
Epoch 2/10
 - 102s - loss: 1.1866 - accuracy: 0.5755 - val_loss: 1.1928 - val_accuracy: 0.6095
Epoch 3/10
 - 79s - loss: 1.0452 - accuracy: 0.6314 - val_loss: 1.0886 - val_accuracy: 0.6376
Epoch 4/10
 - 79s - loss: 0.9474 - accuracy: 0.6668 - val_loss: 1.0094 - val_accuracy: 0.6708
Epoch 5/10
 - 78s - loss: 0.8681 - accuracy: 0.6933 - val_loss: 0.9403 - val_accuracy: 0.6945
Epoch 6/10
 - 78s - loss: 0.7903 - accuracy: 0.7223 - val_loss: 0.9062 - val_accuracy: 0.7071
Epoch 7/10
 - 77s - loss: 0.7267 - accuracy: 0.7458 - val_loss: 0.8628 - val_accuracy: 0.7163
Epoch 8/10
 - 76s - loss: 0.6628 - accuracy: 0.7670 - val_loss: 0.8546 - val_accuracy: 0.7113
Epoch 9/10
 - 76s - loss: 0.6087 - accuracy: 0.7879 - val_loss: 0.8117 - val_accuracy: 0.7320
Epoch 10/10
 - 77s - loss: 0.5528 - accuracy: 0.8057 - val_loss: 0.7743 - val_accuracy: 0.7445


In [20]:
score = model.evaluate(x_img_test_normalize ,y_label_test_OneHot)

10000/10000 [==============================] - 5s 511us/step


In [21]:
score[1]

0.7297000288963318

In [23]:
predict = model.predict_classes(x_img_test_normalize)

In [24]:
pd.crosstab(y_label_test.reshape(-1) , predict , rownames = ['label'] , colnames = ['predict'])

predict,0,1,2,3,4,5,6,7,8,9
label,,,,,,,,,,
0,777,13,33,21,14,7,8,7,86,34
1,18,816,7,14,2,3,7,2,42,89
2,66,4,619,87,75,51,42,29,17,10
3,25,11,66,611,50,139,44,32,11,11
4,13,4,94,86,623,32,57,74,11,6
5,11,3,54,223,38,592,18,47,5,9
6,6,9,44,84,22,17,804,4,7,3
7,14,5,30,42,42,47,5,807,0,8
8,41,31,11,17,6,9,4,5,862,14


In [57]:
#loss = MSE
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
train_history = model.fit(x=x_img_train_normalize,y=y_label_train_OneHot,validation_split=0.2,epochs=10,batch_size=200,verbose=2)


Train on 40000 samples, validate on 10000 samples
Epoch 1/10
 - 78s - loss: 0.0708 - accuracy: 0.4223 - val_loss: 0.0652 - val_accuracy: 0.5150
Epoch 2/10
 - 75s - loss: 0.0566 - accuracy: 0.5665 - val_loss: 0.0553 - val_accuracy: 0.6135
Epoch 3/10
 - 77s - loss: 0.0507 - accuracy: 0.6195 - val_loss: 0.0525 - val_accuracy: 0.6300
Epoch 4/10
 - 75s - loss: 0.0465 - accuracy: 0.6562 - val_loss: 0.0488 - val_accuracy: 0.6602
Epoch 5/10
 - 76s - loss: 0.0430 - accuracy: 0.6868 - val_loss: 0.0457 - val_accuracy: 0.6809
Epoch 6/10
 - 76s - loss: 0.0396 - accuracy: 0.7129 - val_loss: 0.0459 - val_accuracy: 0.6657
Epoch 7/10
 - 74s - loss: 0.0367 - accuracy: 0.7377 - val_loss: 0.0432 - val_accuracy: 0.6906
Epoch 8/10
 - 73s - loss: 0.0337 - accuracy: 0.7634 - val_loss: 0.0402 - val_accuracy: 0.7141
Epoch 9/10
 - 73s - loss: 0.0311 - accuracy: 0.7829 - val_loss: 0.0403 - val_accuracy: 0.7108
Epoch 10/10
 - 77s - loss: 0.0288 - accuracy: 0.7998 - val_loss: 0.0397 - val_accuracy: 0.7152


In [ ]:
#loss = binary_crossentropy
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
train_history = model.fit(x=x_img_train_normalize,y=y_label_train_OneHot,validation_split=0.2,epochs=10,batch_size=200,verbose=2)


Train on 40000 samples, validate on 10000 samples
Epoch 1/10
 - 74s - loss: 0.2598 - accuracy: 0.9053 - val_loss: 0.2390 - val_accuracy: 0.9171
Epoch 2/10
 - 80s - loss: 0.2011 - accuracy: 0.9217 - val_loss: 0.2038 - val_accuracy: 0.9262
Epoch 3/10
 - 79s - loss: 0.1818 - accuracy: 0.9290 - val_loss: 0.1902 - val_accuracy: 0.9340
Epoch 4/10
 - 79s - loss: 0.1665 - accuracy: 0.9349 - val_loss: 0.1810 - val_accuracy: 0.9345
Epoch 5/10
 - 78s - loss: 0.1565 - accuracy: 0.9390 - val_loss: 0.1688 - val_accuracy: 0.9416
Epoch 6/10
 - 79s - loss: 0.1471 - accuracy: 0.9421 - val_loss: 0.1595 - val_accuracy: 0.9432
Epoch 7/10
 - 79s - loss: 0.1376 - accuracy: 0.9461 - val_loss: 0.1582 - val_accuracy: 0.9454
Epoch 8/10
 - 78s - loss: 0.1318 - accuracy: 0.9486 - val_loss: 0.1483 - val_accuracy: 0.9474
Epoch 9/10
 - 77s - loss: 0.1224 - accuracy: 0.9523 - val_loss: 0.1486 - val_accuracy: 0.9460
Epoch 10/10


In [10]:
score = model.evaluate(x_img_test_normalize ,y_label_test_OneHot )

10000/10000 [==============================] - 5s 483us/step


In [11]:
print(score[1])

0.9497103691101074


In [12]:
predict = model.predict_classes(x_img_test_normalize)

In [13]:
predict[:20]

array([3, 8, 8, 8, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6])

In [16]:
import pandas as pd
pd.crosstab(y_label_test.reshape(-1) , predict , rownames = ['label'] , colnames = ['predict'])

predict,0,1,2,3,4,5,6,7,8,9
label,,,,,,,,,,
0,799,18,25,19,15,6,14,12,61,31
1,15,867,6,15,3,10,9,4,18,53
2,70,4,492,71,125,98,90,30,11,9
3,22,11,40,514,68,211,85,29,8,12
4,13,4,32,52,763,36,46,43,9,2
5,15,2,11,159,52,683,35,34,3,6
6,5,10,18,53,40,21,841,3,6,3
7,12,5,14,38,62,73,9,775,4,8
8,45,50,9,17,9,12,10,5,820,23
